Implemented hit selection, events scatter, ion heatmap, electrons time of flight and ions time of flight

Additions:
+ merging multiple runs
+ spatial selection view
+ calibration choice of lines with corresponding zoomed-in views
+ calibration curve fit and apply with mq column

# Imports and functions

In [ ]:
import pandas as pd
import xarray as xr
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
#import ipywidgets as widgets
#from IPython.display import display

In [ ]:
def read(runid):
    'Read the preprocessed data of run with ID runid saved in the h5 file with a corresponding name'
    'Outputs dataframes per event, per pulse, and xarrays etof, pnccd in that order'
    
    filename = '../preprocess/datarun' + str(runid) + '.h5'
    
    dfevent = pd.read_hdf(filename, 'dfevent')
    dfpulse = pd.read_hdf(filename, 'dfpulse')
    
    etof = xr.open_dataarray(filename, group="etof")
    pnccd = xr.open_dataarray(filename, group="pnccd")
    
    return dfevent, dfpulse, etof, pnccd



def events_selection(runs,lower_threshold,upper_threshold):
    'Reads one or multiple runs from h5 files'
    'Makes a pulse selection based on the number of events per pulse between the defined thresholds'
    'If multiple runs are passed, will merge the runs, once hit selected'
    
    
    dataframes = dict()
    
    for run in runs:
        
        dfevent, dfpulse, etof, pnccd = read(run)
        
        selected_dfpulse = dfpulse[lower_threshold < dfpulse.nevents_pulse][dfpulse.nevents_pulse < upper_threshold]
        selected_dfevent = dfevent[dfevent.pulseId.isin(selected_dfpulse.pulseId)]
        selected_etof = etof.sel(pulseId=etof.coords['pulseId'].isin(selected_dfpulse.pulseId))
        
        dataframes[run] = selected_dfevent, selected_dfpulse, selected_etof
        
        plt.figure()
        plt.scatter(dfpulse.pulseId,dfpulse.nevents_pulse,label='All pulses')
        plt.scatter(selected_dfpulse.pulseId,selected_dfpulse.nevents_pulse,c='r',label='Selected pulses')
        plt.xlabel('Pulse ID')
        plt.ylabel('Number of events per pulse')
        plt.legend()
        plt.title(f'Events per pulse with respect to pulse ID for run {run}')
        plt.show()

        
    selected_dfevents = list()
    selected_dfpulses = list()
    selected_etofs = list()
        
    for key, values in dataframes.items():
        
        selected_dfevents.append(values[0])
        selected_dfpulses.append(values[1])
        selected_etofs.append(values[2])
        
    
    merged_dfevent = pd.concat(selected_dfevents)
    merged_dfevent.reset_index(drop=True, inplace=True)
    
    merged_dfpulse = pd.concat(selected_dfpulses)
    merged_dfpulse.reset_index(drop=True, inplace=True)
    
    merged_etof = xr.concat(selected_etofs, dim='pulseId')
    
    
    print(f"Number of pulses selected across {len(runs)} run(s): {len(merged_dfpulse)}")

    
    return merged_dfevent, merged_dfpulse, merged_etof



def heatmap(dfevent):
    'Creates heatmap of the ions hits, based on a dfevent dataframe'
    
    counts_df = dfevent.groupby(['x', 'y']).size().reset_index(name='count')
    heatmap_data = counts_df.pivot(index='y', columns='x', values='count')
    
    plt.figure()
    ax = sns.heatmap(heatmap_data, cmap='viridis',cbar_kws={'label': 'Number of events'})
    plt.title('Ion heatmap')
    plt.show()
    

    
def ion_tof(dfevent):
    'Plots ion time of flight data using dfevent dataframe'
    
    hist, bin_edges = np.histogram(dfevent.tof, bins=250000)
    hist1 = hist[:1500]
    bin_edges1 = bin_edges[:1501]
    
    plt.figure()
    plt.plot(bin_edges1[:-1], hist1)
    plt.xlabel('Time of flight (s)')
    plt.ylabel('Number of hits per bin')
    plt.title('Ions time of flight')
    plt.show()   
    
    
    
def e_tof(etof):
    'Plots electron time of flight data using etof xarray data'
    
    TIME_BETWEEN_PULSES = 3.54462e-6
    CHANNELS_PER_PULSE = 14080
    channel_time = TIME_BETWEEN_PULSES/CHANNELS_PER_PULSE
    
    xaxis = np.arange(14080)*channel_time
    avg_selected_etof = -np.mean(etof, axis=0)
    
    plt.figure()
    plt.plot(xaxis,avg_selected_etof/max(avg_selected_etof))
    plt.xlabel('Time of flight (s)')
    plt.ylabel('Normalized signal')
    plt.title('Electrons time of flight')
    plt.show()